In [1]:
import numpy as np
import pandas as pd
from astropy.table import Table, vstack
from matplotlib import pyplot as plt
from glob import glob
from astropy.stats import LombScargle
from scipy import stats
from scipy.optimize import curve_fit
import warnings
from TESStools import *
import celerite
from celerite import terms
from scipy.optimize import minimize
from tqdm.notebook import tqdm

In [2]:
fyps = pd.read_csv('fyps.csv')

In [3]:
header = ''

for i, star in tqdm(fyps.iterrows(),total=len(fyps)):
    
    fname = star['CommonName']
    print(fname)
    tic = star['TIC ID']
    
    lc, lc_smooth = lc_extract(get_lc_from_id(tic), smooth=128)
    time, flux, err = lc['Time'], lc['Flux'], lc['Err']
    
    good_fs, good_amps, good_phases, good_snrs, good_peaks = prewhiten(time, flux, err)
    
    header += r"""\begin{{deluxetable*}}{{lcccccccc}}
\tabletypesize{{\footnotesize}}
\tablecaption{{Unique frequencies, amplitudes, phases, and formal errors for {0} found via prewhitening. For each frequency, we specify the SNR as defined in text, and the height of the associated peak in the RPS at that stage of prewhitening.\label{{tab:{0}_freqs}}}}
\tablehead{{\colhead{{Prewhitening Stage}} & \colhead{{$f_j$}} & \colhead{{$\epsilon(f_j)$}} & \colhead{{$A_j$}} & \colhead{{$\epsilon(A_j)$}} & \colhead{{$\phi_j$}} &  \colhead{{$\epsilon(\phi_j)$}} & \colhead{{SNR}} & \colhead{{RPS Peak Height}}\\
\colhead{{$j$}} & \colhead{{[day$^{{-1}}$]}} & \colhead{{[day$^{{-1}}$]}} & \colhead{{[ppt]}} & \colhead{{[ppt]}} & \colhead{{[radians]}} & \colhead{{[radians]}} & \colhead{{}} & \colhead{{}} }} 
\startdata
""".format(fname)
    for j,(f,a,p,s,pk) in enumerate(zip(good_fs,good_amps,good_phases,good_snrs,good_peaks)):
        header += f"{int(j)} & ${f[0]:.8f}$ & ${f[1]:.8f}$ & ${a[0]*1e3:.8f}$ & ${a[1]*1e3:.8f}$ & ${p[0]:.4f}$ & ${p[1]:.4f}$ & {s:.4f} & {pk:.4f} \\\\ \n".replace('e','\\times10^{')
    header.rstrip()
    header.rstrip('\\')
    header.rstrip()
    header += r"""\enddata
\end{deluxetable*}

"""
    
with open("appendix_deluxetable.txt", "w") as text_file:
        text_file.write(header)

HD 269953
10
Found 10 frequencies
10 unique frequencies
HD 269110
Found 8 frequencies
8 unique frequencies
HD 268687
Found 8 frequencies
8 unique frequencies
HD 269840
10
Found 16 frequencies
16 unique frequencies
HD 269902
Found 5 frequencies
5 unique frequencies

